### 사용자 입장에서 해야 할 step !
- G마켓 BEST상품의 1위부터 10위까지의 데이터를 수집
- 중요한 것 !
    - 첫 번째에 있는 상품을 클릭한 다음에 해당 상품의 이름과 가격 정보를 수집
    - 뒤로 간 다음 두 번째에 있는 상품을 클릭, 수집 똑같은 말을 반복 중
    - while ? for ? 어떤 것이 적합 ? -> for문 / 10번 정해져 있기 때문

In [1]:
# selenium 라이브러리 로딩

# 브라우저 역할 - 컴퓨터용 브라우저
from selenium import webdriver as wb

# 키보드 역할 (Keys K대문자 !!) - 컴퓨터용 키보드
from selenium.webdriver.common.keys import Keys

# 구분자 역할 (By B대문자 !!) - 선택자를 구분
from selenium.webdriver.common.by import By

# 수집한 이후 데이터프레임까지 만들어볼 예정
import pandas as pd

# 지연시간 설정
import time

In [2]:
driver = wb.Chrome()
url = 'https://corners.gmarket.co.kr/bestsellers'
driver.get(url)

#### 반복문을 시작하기 전 하나의 프로세스를 완벽하게 짜보기
#### 잘 완성되면 반복문 안에 프로세스를 통째로 넣어주기만 하면 됨!
1. 상품의 이미지를 클릭 (화면 변경)
> 사용자는 첫 번째 상품을 알 수 있지만 컴퓨터는 알 수 없다 !  
> 어떤 태그로 구성되어 있는지 검사해서 알려줘야 함
2. 상품의 이름과 가격을 수집
3. 페이지 뒤로가기 (화면 변경)

In [4]:
# 1. 상품의 이미지를 클릭

# 1-1. 이미지를 클릭하기 위해서 이미지 태그들을 알려줘야 함
imgs = driver.find_elements(By.CLASS_NAME, 'lazy')

# 1-2. 이미지 클릭
imgs[0].click()

# 요청을 복수개로 했으니 응답도 복수개이다 -> [리스트]

In [9]:
# 2-1. 상품명 수집

# 조회하고자 하는 요소가 class를 가지고 있을 경우에는
# 먼저 ! elements(복수)로 요청해본다.
# 그 다음에 데이터가 한 개만 넘어온다면, element(단일)로 수정해서 활용하면 편하다 !
title = driver.find_element(By.CLASS_NAME, 'itemtit')
title.text

'1+1 아모스 컬링 에센스 2X 150ml 당일발송 (최종 12680원)'

In [11]:
# 2-2. 상품 가격 수집
# price 변수에 담아서 price.text 컨텐츠 추출

price = driver.find_element(By.CLASS_NAME, 'price_real')
price.text

'16,900원'

In [12]:
# 페이지 뒤로가기
driver.back()

In [13]:
# 1번부터 5번까지 데이터 수집

for i in range(5) :
    
    imgs = driver.find_elements(By.CLASS_NAME, 'lazy')
    imgs[i].click()
    
    title = driver.find_element(By.CLASS_NAME, 'itemtit')
    title.text
    
    price = driver.find_element(By.CLASS_NAME, 'price_real')
    price.text
    
    driver.back()
    